<a href="https://colab.research.google.com/github/mubarakdalvi/Data-Science-Projects/blob/main/CC_Fraud_Detection_Using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faker

In [ ]:
!pip install maxminddb-geolite2

In [ ]:
! pip install random_ip_generator

In [ ]:
! pip install pip pycountry

In [ ]:
import random
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime,time
from geolite2 import geolite2
import pycountry
from random_ip_generator import random_ip_for_country
import ipaddress
import re
from sklearn.preprocessing import LabelEncoder

In [ ]:
fake = Faker('en-IN')

have to map one credit card to one user there will be not duplicate , we can use set()

Card Number, Expiration Date, CVV/CVC, Cardholder Name, Billing Address, normal ip address, suspisious ip address, amount frequency, high  transaction amount,suspicious sites where user had visisted, whether user have enabled 2fa or not, customer alert, card type

In [ ]:
Visa_creditCard = set()
MasterCard_creditCard = set()
Amex_creditCard = set()
Discover_creditCard = set()
def generate_card_number():
    i = 0
    while i < 4000000:
        card_type = random.choice(['Visa', 'MasterCard', 'Amex', 'Discover'])
        if card_type == 'Visa':
            Visa_creditCard.add(random.randint(4000000000000000, 4999999999999999))
        elif card_type == 'MasterCard':
            MasterCard_creditCard.add(random.randint(5000000000000000, 5999999999999999))
        elif card_type == 'Amex':
            Amex_creditCard.add(random.choice([random.randint(400000000000000, 499999999999999), random.randint(700000000000000, 799999999999999)]))
        elif card_type == 'Discover':
            Discover_creditCard.add(random.randint(6000000000000000, 6999999999999999))
        i += 1
print(generate_card_number())

In [ ]:
countries = [country.name for country in pycountry.countries]
Country_ip = []
country_name = []
i = 0
while i < 1000:
  try:
    sus_countries = random.choice(countries)
    ip_from_sus_country = random_ip_for_country(pycountry.countries.get(name=sus_countries).alpha_2)
    ip_str = str(ip_from_sus_country)
    Country_ip.append(ip_str),country_name.append(sus_countries)
    i += 1
  except ValueError:
    print(f'Error Code : {sus_countries}')
print(len(Country_ip),Country_ip,len(country_name),country_name)

In [ ]:
transaction_day = []
def if_leap():
  for i in range(1,1001):
    month,year = random.randint(1,12), random.randint(2023,2024)
    if month == 2:
      if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0):
        day = random.randint(1,29)
      else:
        day = random.randint(1,28)
    elif month in [4,6,9,11]:
      day = random.randint(1,30)
    else:
      day = random.randint(1,31)
    date = datetime(year,month,day)
    date_app = date.strftime('%d/%m/%Y')
    transaction_day.append(date_app)
print(if_leap())

In [ ]:
suspected_users = []
def is_suspicious():
  try:
    geo = geolite2.reader()
    for  i in range(1000):
      ip_india = f'{59}.{181}.{20}.{random.randint(0,255)}'
      india_country = [geo.get(ip_india)['country']['names']['en']] if geo.get(ip_india) and 'country' in geo.get(ip_india) else []
      sus_countries = random.choice(country_name)
      ip_from_sus_country = random.choice(Country_ip)
      Billing_Address = fake.address()
      provided_address = fake.address()
      Two_FA = random.choice(['yes','no'])
      sus_transaction_time = random.choice([time(random.randint(2,5),random.randint(0,59),random.randint(0,59)).strftime('%I:%M:%S %p')])
      users_frequent_timeofpurchse = time(random.randint(19,23),random.randint(0,59),random.randint(0,59)).strftime('%I:%M:%S: %p')
      high_transaction_after_suspicion = random.choice(['yes','no'])
      customer_alert = random.choice(['yes','no'])
      trandaction_location = fake.address()
      current_location = fake.address()
      using_vpn = random.choice(['yes','no'])
      failed_logins = random.randint(0, 5)
      otp = fake.random_number(digits=6,fix_len=True)
      transaction = random.choice(transaction_day)
      transaction_date = transaction
      is_sus = []
      if ip_india != ip_from_sus_country and using_vpn == 'yes':
        if sus_transaction_time != users_frequent_timeofpurchse:
            if high_transaction_after_suspicion == 'yes' and Two_FA == 'no':
                if 3 <= failed_logins <= 5:
                    is_sus.append('yes')
                else:
                    is_sus.append('no')
            else:
              is_sus.append('no')
        else:
          is_sus.append('no')
      else:
        is_sus.append('no')
      suspected_users.append({
          'Billing_Address' : Billing_Address,
          'provided_address' : provided_address,
          'ip_india' : ip_india,
          'india_country' : india_country,
          'ip_from_sus_country' : ip_from_sus_country,
          'sus_countries' : sus_countries,
          'Two_FA' : Two_FA,
          'sus_transaction_time' : sus_transaction_time,
          'users_frequent_timeofpurchse' : users_frequent_timeofpurchse,
          'high_transaction_after_suspicion' : high_transaction_after_suspicion,
          'customer_alert' : customer_alert,
          'trandaction_location' : trandaction_location,
          'current_location' : current_location,
          'using_vpn' : using_vpn,
          'failed_logins' : failed_logins,
          'otp' : otp,
          'is_sus' : is_sus,
          'transaction_date' : transaction
          })
  except Exception as e:
    print(f'Error Code : {e}')
  return suspected_users
print(is_suspicious())

In [ ]:
creditcard_info = []
def creditcard_data():
  i = 0
  while i < 1000:
    card_type = random.choice(['Visa','MasterCard','Amex','Discover'])
    Expiration_Date = fake.credit_card_expire(start='now',end='+10y',date_format='%m/%y')
    CVV = fake.random_number(digits=3,fix_len=True)
    Cardholder_Name = fake.name()
    Cards = None
    if card_type == 'Visa':
      Cards = random.choice(list(Visa_creditCard))
    elif card_type == 'MasterCard':
      Cards = random.choice(list(MasterCard_creditCard))
    elif card_type == 'Amex':
      Cards = random.choice(list(Amex_creditCard))
    elif card_type == 'Discover':
      Cards = random.choice(list(Discover_creditCard))
    creditcard_info.append({
        'Cardholder_Name' : Cardholder_Name,
        'Cards' : Cards,
        'card_type' : card_type,
        'CVV' : CVV,
        'Expiration_Date' : Expiration_Date,
        })
    i += 1
  return creditcard_info
print(creditcard_data())

In [ ]:
sus = pd.DataFrame(creditcard_info)
sus.to_csv('customer_data')

In [ ]:
bus = pd.DataFrame(suspected_users)
bus.to_csv('parameter')

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [ ]:
df_customer_data = pd.read_csv('customer_data',index_col=0)
df_customer_data.head()

In [ ]:
df_customer_data.info()

In [ ]:
df_customer_data.corr()

In [ ]:
df_parameter = pd.read_csv('parameter',index_col=0)

In [ ]:
df_parameter.info()

In [ ]:
india_country = df_parameter['india_country'].str.replace("\\['",'').str.replace("\\']",'')

In [ ]:
df_parameter['india_country'] = india_country

In [ ]:
df_parameter.head()

In [ ]:
is_sus = df_parameter['is_sus'].str.replace("\\['",'').str.replace("\\']",'')

In [ ]:
df_parameter['is_sus'] = is_sus

In [ ]:
df_parameter.head()

In [ ]:
df_parameter['is_sus'].value_counts()

In [ ]:
df = df_customer_data.join(df_parameter,how='inner')

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
def extract(col):
  if isinstance(col, str):
    match = re.search('\d{6}$',col)
    if match is not None:
      return match.group()
    return col
df['Billing_Address'] = df['Billing_Address'].apply(extract)

In [ ]:
def extract(col):
  if isinstance(col, str):
    match = re.search('\d{6}$',col)
    if match is not None:
      return match.group()
    return col
df['provided_address'] = df['provided_address'].apply(extract)

In [ ]:
def extract(col):
  if isinstance(col, str):
    match = re.search('\d{6}$',col)
    if match is not None:
      return match.group()
    return col
df['trandaction_location'] = df['trandaction_location'].apply(extract)

In [ ]:
def extract(col):
  if isinstance(col, str):
    match = re.search('\d{6}$',col)
    if match is not None:
      return match.group()
    return col
df['current_location'] = df['current_location'].apply(extract)

In [ ]:
check = df['Cardholder_Name'].value_counts()

In [ ]:
df[df['Cardholder_Name'].isin(check[check >= 2].index)]

In [ ]:
'''df['transaction_date'] = df['transaction_date'].str.replace("\['",'').str.replace("\']",'')'''

In [ ]:
'''df['day'],df['month'],df['year'] = df['transaction_date'].str.split('/',2).str'''

In [ ]:
df.corr()

In [ ]:
def impute(col):
  if isinstance(col,str):
    if col == 'India':
      return 1
    else:
      return col
df['india_country'] = df['india_country'].apply(impute)

In [ ]:
'''df['Expiration_Date'] = df['Expiration_Date'].astype(str)'''

In [ ]:
'''_,df['Expiration_Date_year'] = df['Expiration_Date'].str.split('/',1).str'''

In [ ]:
le = LabelEncoder()

In [ ]:
transformed_Cards = le.fit_transform(df['Cards'])
df['Cards'] = transformed_Cards

In [ ]:
transformed_card_type = le.fit_transform(df['card_type'])
df['card_type'] = transformed_card_type

In [ ]:
transformed_Expiration_Date = le.fit_transform(df['Expiration_Date'])
df['Expiration_Date'] = transformed_Expiration_Date

In [ ]:
transformed_Billing_Address = le.fit_transform(df['Billing_Address'])
df['Billing_Address'] = transformed_Billing_Address

In [ ]:
transformed_provided_address = le.fit_transform(df['provided_address'])
df['provided_address'] = transformed_provided_address

In [ ]:
transformed_ip_india = le.fit_transform(df['ip_india'])
df['ip_india'] = transformed_ip_india

In [ ]:
transformed_ip_from_sus_country = le.fit_transform(df['ip_from_sus_country'])
df['ip_from_sus_country'] = transformed_ip_from_sus_country

In [ ]:
transformed_sus_countries = le.fit_transform(df['sus_countries'])
df['sus_countries'] = transformed_sus_countries

In [ ]:
transformed_Two_FA = le.fit_transform(df['Two_FA'])
df['Two_FA'] = transformed_Two_FA

In [ ]:
transformed_sus_transaction_time = le.fit_transform(df['sus_transaction_time'])
df['sus_transaction_time'] = transformed_sus_transaction_time

In [ ]:
transformed_users_frequent_timeofpurchse = le.fit_transform(df['users_frequent_timeofpurchse'])
df['users_frequent_timeofpurchse'] = transformed_users_frequent_timeofpurchse

In [ ]:
transformed_failed_logins = le.fit_transform(df['failed_logins'])
df['failed_logins'] = transformed_failed_logins

In [ ]:
transformed_high_transaction_after_suspicion = le.fit_transform(df['high_transaction_after_suspicion'])
df['high_transaction_after_suspicion'] = transformed_high_transaction_after_suspicion

In [ ]:
transformed_customer_alert = le.fit_transform(df['customer_alert'])
df['customer_alert'] = transformed_customer_alert

In [ ]:
transformed_trandaction_location = le.fit_transform(df['trandaction_location'])
df['trandaction_location'] = transformed_trandaction_location

In [ ]:
transformed_current_location = le.fit_transform(df['current_location'])
df['current_location'] = transformed_current_location

In [ ]:
transformed_using_vpn = le.fit_transform(df['using_vpn'])
df['using_vpn'] = transformed_using_vpn

In [ ]:
transformed_failed_logins = le.fit_transform(df['failed_logins'])
df['failed_logins'] = transformed_failed_logins

In [ ]:
transformed_otp = le.fit_transform(df['otp'])
df['otp'] = transformed_otp

In [ ]:
transformed_is_sus = le.fit_transform(df['is_sus'])
df['is_sus'] = transformed_is_sus

In [ ]:
transformed_transaction_date = le.fit_transform(df['transaction_date'])
df['transaction_date'] = transformed_transaction_date

In [ ]:
df.drop('Cardholder_Name',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.corr()

In [ ]:
x = df.drop('is_sus',axis=1)

In [ ]:
x

In [ ]:
y = df['is_sus']

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=31)

In [ ]:
x_train

In [ ]:
x_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr1 = LogisticRegression()
lr2 = LogisticRegression(C = 100, penalty ='l2', solver = 'liblinear')
lr3 = LogisticRegression()

In [ ]:
lr1.fit(x_train,y_train)

In [ ]:
y_pred1 = lr1.predict(x_test)

In [ ]:
lr2.fit(x_train,y_train)

In [ ]:
y_pred2 = lr2.predict(x_test)

In [ ]:
lr3.fit(x_train,y_train)

In [ ]:
y_pred3 = lr3.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
print(f'y_pred1 = {accuracy_score(y_test,y_pred1)}')

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
print(f'y_pred2 = {accuracy_score(y_test,y_pred2) * 100}%')

In [ ]:
confusion_matrix(y_test,y_pred2)

In [ ]:
print(f'y_pred3 = {accuracy_score(y_test,y_pred3)}')

In [ ]:
confusion_matrix(y_test,y_pred3)

In [ ]:
param_grid = {'penalty': ['l1', 'l2'], 'solver': ['liblinear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
grid_search = GridSearchCV(lr1,param_grid,cv=5)

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
cross_val_score(lr1,x,y,cv=5,scoring='accuracy').mean() * 100